## Imports

In [ ]:
import evalseg
from evalseg.metrics import MME,MultiClassMetric
import numpy as np
import pandas as pd
from ipywidgets import interact, interact_manual, IntSlider
import os,glob
import auto_profiler
import json

## Variables

In [ ]:
root_data = '../datasets'
only_2d_slice=True
out_root='../out'
os.makedirs(out_root,exist_ok=True)

# Metrics

In [ ]:
def get_metrics(num_classes)->evalseg.metrics.MultiMetric:
    metrics = {
        # 'mme': evalseg.metrics.MME(),
        'hd': evalseg.metrics.HD(),
        'voxel': evalseg.metrics.Voxel(),
        'nsd t=1': evalseg.metrics.NSD(tau=1),
        'nsd t=5': evalseg.metrics.NSD(tau=5),

        # evalseg.metrics.BD,
    }
    from evalseg.metrics import MultiClassMetric, MultiMetric
    return MultiMetric(metrics_dic={m: MultiClassMetric(metrics[m], num_classes)for m in metrics})


## Run Metric over all datasets 

# For 3D

In [ ]:
evalseg.reload()




with auto_profiler.Profiler(filterExternalLibraries=False,depth=20):
 for dataset_name in sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')]):
    # if 'Livre' in dataset_name:
    #     continue
    dataset = evalseg.io.Dataset(f'{root_data}/{dataset_name}')
    for case in sorted(dataset.get_available_ids()):
        out_path=f'{out_root}/{dataset_name}-{case}'
        if len(glob.glob(f'{out_path}-*')) >= (3 if only_2d_slice else 1):
            continue

        # dataset.load_all_of_case(case)
        gto = dataset.get_groundtruth(case)
        # cto = dataset.get_CT(case)
        # if cto is None:
        #     continue

        metric = get_metrics(dataset.num_labels)
        
        metric.set_reference(gto)
        preds = {p: dataset.get(p, case) for p in dataset.get_prediction_methods()}
        res = metric.evaluate_multi(preds,parallel=0)
        print(res)
        with open(f'{out_path}.json', 'w') as f:
            json.dump(res,f,indent=4)
        
        
    break


In [ ]:
import evalseg
evalseg.metrics.MME.__class__

# For Max Slice (2D)

In [ ]:
evalseg.reload()
# with auto_profiler.Profiler(filterExternalLibraries=False,depth=10):
for dataset_name in sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')]):
    if 'Livre' in dataset_name:continue
    dataset = evalseg.io.NibDataset(f'{root_data}/{dataset_name}')
    for case in sorted(dataset.get_available_ids()):
        if len(glob.glob(f'{out_root}/{dataset_name}-{case}-*')) >= (3 if only_2d_slice else 1):
            continue

        # dataset.load_all_of_case(case)
        gto, gto_voxelsize = dataset.get_groundtruth(case)
        cto, cto_voxelsize = dataset.get_CT(case)
        if cto is None: continue
        
        if only_2d_slice:
            for c in range(1, dataset.num_labels):
                gto_tmp=gto == c
                gtmax = (gto_tmp.sum(axis=2).sum(axis=1).argmax(), gto_tmp.sum(axis=2).sum(axis=0).argmax(), gto_tmp.sum(axis=1).sum(axis=0).argmax())
                axes = ['x', 'y', 'z'] 
                for ax in axes:
                    axi = {'all': -1, 'x': 0, 'y': 1, 'z': 2}[ax]
                    frame = -1 if ax == 'all' else gtmax[axi]
                    
                    gt, voxelsize = evalseg.geometry.slice(gto, gto_voxelsize, axi, [frame])
                    ct, _ = evalseg.geometry.slice(cto, None, axi, [frame])
                    preds = {p: evalseg.geometry.slice(dataset.get_prediction(p, case)[0], None, axi, [frame])[0] == c
                            for p in dataset.get_prediction_methods()}
                    preds = {'GroundTruth': gt, **preds}

                    out_path = f'{out_root}/{dataset_name}-{case}'
                    
                    out_path = f'{out_path}-{ax}-{frame}-for{c}'
                    ctlbl = f"{dataset_name.split('_')[1]} {case} {ax}={frame}"
                    evalseg.ui.multi_plot_2d(ct, gt,  preds,
                                            spacing=voxelsize, col=6, ctlbl=ctlbl, z_titles=[frame],
                                            dst=f'{out_path}-allct.png', show=False)
                    
                    print(out_path)

                    # mme = evalseg.metrics.MultiClassMetric(evalseg.metrics.MME, dataset.num_labels)
                    metric = get_metrics(dataset.num_labels)
        
                    metric.set_reference(gto, gto_voxelsize)
                    
                    res = metric.evaluate_multi(preds,parallel=0)
                    print(res)
                    with open(f'{out_path}.json', 'w') as f:
                        json.dump(res,f,indent=4)
                    
                    k = list(res['mme'].keys())[0]
                    res = {'ignore it a': res['mme'][k], 'ignore it b': res['mme'][k], **res}
                    evalseg.ui.plot_metric_multi(res['mme'], name=dataset_name, dst=f'{out_path}-metric{c}.png', show=False, col=6)
                    
                    evalseg.ui.img.concat(glob.glob(f'{out_path}-[!res]*.png'), out=f'{out_path}-res-{c}.png')
                    [os.remove(f) for f in glob.glob(f'{out_path}-[!res]*.png')]


In [ ]:
evalseg.reload()
with auto_profiler.Profiler(filterExternalLibraries=False,depth=10):
    metric = get_metrics(dataset.num_labels)
            
    metric.set_reference(gt, voxelsize)
    res = metric.evaluate_multi(preds,parallel=0)
    print(res)

In [ ]:
evalseg.reload()
res = metric.evaluate_multi({'CE':preds['CE']},parallel=0)
print(res)

In [ ]:
str({'sadsadsad':np.zeros(10)})[0:1000]

In [ ]:
data['GroundTruth'].shape